>**기출문제 제3회**
>>**01 작업형 제1유형**

## 옵션 설정

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # 모든 칼럼이 출력되게 조절

In [2]:
import pandas as pd 

data = pd.read_csv('3_1.csv')
data.head()

,city,homeType,latitude,longitude,garageSpaces,hasSpa,yearBuilt,numOfPatioAndPorchFeatures,lotSizeSqFt,avgSchoolRating,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,priceRange
0,austin,Single Family,30.197647,-97.816811,2,False,1981,0,9757.0,4.000000,13,NaN,3,350000-450000
1,austin,Single Family,30.329187,-97.752731,2,False,1951,0,10846.0,7.000000,16,NaN,4,650000+
2,austin,Single Family,30.313305,-97.699669,0,False,1954,0,9016.0,5.000000,15,NaN,3,350000-450000
3,austin,Single Family,30.156321,-97.748291,2,False,2008,0,5662.0,5.333333,15,NaN,3,0-250000
4,austin,Townhouse,30.366995,-97.753548,2,False,1984,0,3049.0,7.000000,16,NaN,2,350000-450000


## 문제 1. 
- 다음 데이터에서 lotSizeSqFt이 큰 top 10을 구하고, top 10 값 중 가장 작은 값으로 해당 값을 대치하시오. 
- 또한, 건축 연도(yearBuilt)가 2000년도 이상인 IoTSizeSqFt의 평균을 구하시오.
- 소수점 첫째 자리에서 반올림, 대치된 IoTSizeSqFt 기준


In [27]:
# 코드 작성
data1 = data.copy()
# print(data.head(1))

# pandas 메서드 중, n~메서드 중, nlargest()
top_10_idx = data1['lotSizeSqFt'].nlargest(10).index
min_top_10 = data1.loc[top_10_idx, 'lotSizeSqFt'].min()
print(min_top_10)
print(data1.loc[top_10_idx, 'lotSizeSqFt'])

# top 10에 해당하는 lotSizeSqFt 필드를 min_tio_10로 대체하세요
data1.loc[top_10_idx, 'lotSizeSqFt'] = min_top_10
data1.loc[412, :]

# 2000년도 이상 건축물의 lotSizeSqFt 평균 계산
mean_lot_size = data1.loc[data1['yearBuilt'] >= 2000, 'lotSizeSqFt'].mean()
print(round(mean_lot_size, 1))

71438.4
412    871200.0
6      225205.2
175    211701.6
97     197326.8
68     175982.4
440     98445.6
399     87120.0
480     80150.4
265     77972.4
71      71438.4
Name: lotSizeSqFt, dtype: float64
12954.9


## 문제 2. 
- 컬럼별 결측치 존재 여부를 확인하고, 결측치가 존재하는 경우 해당 컬럼의 중앙값으로 결측치를 대치하시오. 
- 결측치 대치 전과 후 표준편차 차이의 절댓값을 구하시오. 
- 소수점 둘째 자리까지 반올림


In [33]:
data2 = data.copy()

# 결측치 존재 여부 확인
# print(data2.isnull().sum())
# 어떤 데이터프레임이 있어도 하나의 코드로 재활용이 가능한 방법이 뭘까? <- 고민해야하는 부분

print(data2.loc[:, 'numOfBathrooms'].std())

0.9893368595360542


In [40]:
# 코드 작성, 결측치가 있는 컬럼 찾기
cols_with_null = data2.columns[data2.isnull().any()].tolist()
print(cols_with_null)

# 결측치 대치하기 전의 표준편차 저장
std_before = data.loc[:, cols_with_null].std()
std_before

['numOfBathrooms']


numOfBathrooms    0.989337
dtype: float64

In [43]:
# 코드 작성
# 2개 이상 컬럼에 각각 서로다른 컬럼의 결측치에 중앙값을 대치
# data2.loc[].fillna()
for col in cols_with_null:
    print("시작")
    print(f"현재 컬럼: {col}")
    median_val = data2.loc[:, col].median()
    print(f"현재 컬럼의 중앙값: {median_val}")
    data2.loc[data2[col].isnull(), col] = median_val

std_after = data2.loc[:, cols_with_null].std()
std_before, std_after

시작
현재 컬럼: numOfBathrooms
현재 컬럼의 중앙값: 3.0


(numOfBathrooms    0.989337
 dtype: float64,
 numOfBathrooms    0.97955
 dtype: float64)

## 문제 3. 
- 평균으로부터 1.5x 표준편차 범위 밖의 값을 이상치로 판단
- MedianStudentsPerTeacher의 이상치를 구하고, 이상치의 개수를 구하시오.
    + 회귀모형을 활용한 이상치 검증

In [46]:
# 코드 작성
# 이상값에 대해서 설명
# 이상치 판별 시, 가장 확실한 방법 : 도메인 기반, 추천 방식: 회귀모형 이상치 판별

data3 = data.copy()

# 이상치 범위 계산 (평균 ± 1.5 * 표준편차)
spt_mean = data3.loc[:, 'MedianStudentsPerTeacher'].mean()
spt_std = data3.loc[:, 'MedianStudentsPerTeacher'].std()

print(spt_mean, spt_std)
lower_bound = spt_mean - 1.5 * spt_std
upper_bound = spt_mean + 1.5 * spt_std

# 이상치 찾기, 조건식 OR 연산자 활용
outliers = data3.loc[(data3.loc[:, 'MedianStudentsPerTeacher'] < lower_bound) |
                     (data3.loc[:, 'MedianStudentsPerTeacher'] > upper_bound)]

# 이상치 개수 출력
print("이상치 개수:", len(outliers))

15.088 1.6738756582968994
이상치 개수: 65


In [47]:
def detect_outliers_by_std(data, column, threshold=1.5):
    if column not in data.columns:
        raise ValueError(f"컬럼 '{column}' 이(가) 데이터에 없습니다.")

    df = data.copy()
    col_mean = df[column].mean()
    col_std = df[column].std()
    
    lower_bound = col_mean - threshold * col_std
    upper_bound = col_mean + threshold * col_std
    
    outliers_df = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    outlier_count = len(outliers_df)
    stats = (round(col_mean, 2), round(col_std, 2), round(lower_bound, 2), round(upper_bound, 2))
    
    return outliers_df, outlier_count, stats


In [49]:
outliers, count, stats = detect_outliers_by_std(data, 'MedianStudentsPerTeacher', threshold=1.5)
print(f"이상치 개수: {count}")
print("통계:", stats)

이상치 개수: 65
통계: (np.float64(15.09), np.float64(1.67), np.float64(12.58), np.float64(17.6))
